**Задание 1.** Анализ производительности CPU-параллельной программы (OpenMP)
Разработайте параллельную программу на C++ с использованием OpenMP для обработки
большого массива данных (например, вычисление суммы, среднего значения и
дисперсии).

**Требуется:**
* реализовать базовую параллельную версию;
* выполнить профилирование программы с использованием omp_get_wtime() и/или
профилировщика (Intel VTune, gprof);

**определить:**
* долю параллельной и последовательной части программы;
* влияние числа потоков на ускорение;
* проанализировать результаты в контексте закона Амдала.

In [2]:
%%writefile omp_task1.cpp

#include <omp.h>                 // OpenMP (omp_get_wtime, omp_set_num_threads)
#include <iostream>              // cout
#include <vector>                // vector
#include <random>                // mt19937, uniform
#include <cmath>                 // fabs
#include <cstdlib>               // atoll

int main(int argc, char** argv)                                          // вход
{
    long long N = 50'000'000;                                            // размер массива по умолчанию (50 млн)
    if (argc > 1) N = std::atoll(argv[1]);                               // если передали N

    if (N <= 0)                                                          // проверка
    {
        std::cout << "N должно быть > 0\n";                              // сообщение
        return 0;                                                        // выход
    }

    std::vector<double> a;                                               // массив данных
    a.resize((size_t)N);                                                 // выделяем память

    std::mt19937 gen(42);                                                // генератор случайных чисел
    std::uniform_real_distribution<double> dist(0.0, 1.0);               // распределение 0..1

    for (long long i = 0; i < N; ++i)                                    // заполняем массив (последовательно)
        a[(size_t)i] = dist(gen);                                        // записали число

    int max_threads = omp_get_max_threads();                             // сколько потоков доступно
    std::cout << "N = " << N << "\n";                                    // печать N
    std::cout << "Max threads available = " << max_threads << "\n\n";    // печать max потоков

    //  ПОСЛЕДОВАТЕЛЬНАЯ ВЕРСИЯ (baseline)
    double t0_seq = omp_get_wtime();                                     // старт времени seq

    double sum_seq = 0.0;                                                // сумма
    for (long long i = 0; i < N; ++i)                                    // цикл
        sum_seq += a[(size_t)i];                                         // суммирование

    double mean_seq = sum_seq / (double)N;                               // среднее

    double var_seq_acc = 0.0;                                            // накопитель дисперсии
    for (long long i = 0; i < N; ++i)                                    // второй проход
    {
        double d = a[(size_t)i] - mean_seq;                              // отклонение
        var_seq_acc += d * d;                                            // квадрат отклонения
    }
    double var_seq = var_seq_acc / (double)N;                            // дисперсия

    double t1_seq = omp_get_wtime();                                     // конец времени seq
    double time_seq = t1_seq - t0_seq;                                   // время seq

    std::cout << "[SEQ] sum=" << sum_seq                                 // вывод суммы
              << " mean=" << mean_seq                                    // вывод среднего
              << " var=" << var_seq                                      // вывод дисперсии
              << " time=" << time_seq << " s\n\n";                       // вывод времени

    //ПАРАЛЛЕЛЬНЫЕ ЗАПУСКИ ДЛЯ РАЗНЫХ ЧИСЕЛ ПОТОКОВ
    // Будем прогонять 1,2,4,8,... до max_threads (степени двойки)
    std::cout << "threads,time_parallel,speedup,efficiency,parallel_fraction(f),serial_fraction(1-f)\n";

    for (int threads = 1; threads <= max_threads; threads *= 2)          // перебор потоков
    {
        omp_set_num_threads(threads);                                    // задаём число потоков

        // Чтобы сравнение было честным, считаем то же самое: sum -> mean -> variance
        double t0 = omp_get_wtime();                                     // старт параллельного замера

        double sum_par = 0.0;                                            // параллельная сумма

        #pragma omp parallel for reduction(+:sum_par) schedule(static)   // параллельный цикл + редукция суммы
        for (long long i = 0; i < N; ++i)                                // по всем элементам
            sum_par += a[(size_t)i];                                     // суммируем

        double mean_par = sum_par / (double)N;                           // среднее

        double var_par_acc = 0.0;                                        // накопитель дисперсии

        #pragma omp parallel for reduction(+:var_par_acc) schedule(static) // параллельный цикл для дисперсии
        for (long long i = 0; i < N; ++i)                                // по всем элементам
        {
            double d = a[(size_t)i] - mean_par;                          // отклонение
            var_par_acc += d * d;                                        // квадрат отклонения
        }

        double var_par = var_par_acc / (double)N;                        // дисперсия

        double t1 = omp_get_wtime();                                     // конец параллельного замера
        double time_par = t1 - t0;                                       // время параллельной версии

        // Проверим корректность (сравним с последовательной, допуск небольшой)
        double eps_mean = std::fabs(mean_par - mean_seq);                // ошибка по mean
        double eps_var  = std::fabs(var_par  - var_seq);                 // ошибка по var

        // Ускорение и эффективность
        double speedup = time_seq / time_par;                            // S = T1 / Tp
        double efficiency = speedup / (double)threads;                   // E = S / p

        // Оценка доли параллельной части по закону Амдала:
        // S(p) = 1 / ( (1-f) + f/p )
        // => f = (1 - 1/S) / (1 - 1/p)
        double f = 0.0;                                                  // доля параллельной части
        if (threads > 1)                                                 // если p>1
        {
            double invS = 1.0 / speedup;                                 // 1/S
            f = (1.0 - invS) / (1.0 - 1.0 / (double)threads);            // формула для f
            if (f < 0.0) f = 0.0;                                        // защита
            if (f > 1.0) f = 1.0;                                        // защита
        }
        else
        {
            f = 0.0;                                                     // при 1 потоке Амдал не оцениваем
        }

        double serial_part = 1.0 - f;                                    // последовательная доля

        std::cout << threads << ","                                      // threads
                  << time_par << ","                                     // time_parallel
                  << speedup << ","                                      // speedup
                  << efficiency << ","                                   // efficiency
                  << f << ","                                            // parallel fraction
                  << serial_part << "\n";                                // serial fraction

        // Можно ещё вывести ошибки, если нужно (но чтобы не засорять таблицу — закомментировано)
        // if (rank == 0) std::cout << "eps_mean=" << eps_mean << " eps_var=" << eps_var << "\n";
        (void)eps_mean;                                                  // чтобы компилятор не ругался, если не выводим
        (void)eps_var;                                                   // то же самое
    }

    return 0;                                                            // конец
}

Overwriting omp_task1.cpp


In [3]:
!g++ -O2 -fopenmp omp_task1.cpp -o omp_task1
!./omp_task1 50000000

N = 50000000
Max threads available = 2

[SEQ] sum=2.49983e+07 mean=0.499966 var=0.0833294 time=0.138123 s

threads,time_parallel,speedup,efficiency,parallel_fraction(f),serial_fraction(1-f)
1,0.137311,1.00591,1.00591,0,1
2,0.0752641,1.83518,0.917589,0.910187,0.0898129


**Задание 2.** Оптимизация доступа к памяти на GPU (CUDA)
Реализуйте ядро CUDA для обработки массива данных, демонстрирующее разные
паттерны доступа к памяти.

**Требуется:**
1. реализовать две версии ядра:
a. с эффективным (коалесцированным) доступом к глобальной памяти;
b. с неэффективным доступом к памяти;
2. измерить время выполнения с использованием cudaEvent;
3. провести оптимизацию за счёт:
a. использования разделяемой памяти;
b. изменения организации потоков;
4. сравнить результаты и сделать выводы о влиянии доступа к памяти на
производительность GPU.

In [7]:
%%writefile task2.cu
// cuda_mem_patterns.cu — Практическая работа №10, Задание 2
// Оптимизация доступа к памяти на GPU (CUDA):
// 1) коалесцированный доступ к global memory
// 2) некоалесцированный (плохой) доступ по stride
// 3) оптимизация с использованием shared memory (tile)
// 4) влияние организации потоков (blockSize)
// Замер времени через cudaEvent

#include <cuda_runtime.h>    // CUDA runtime
#include <iostream>          // cout
#include <vector>            // vector
#include <cmath>             // fabs
#include <cstdlib>           // atoi, exit
#include <algorithm>         // max

// Макрос проверки ошибок CUDA
#define CUDA_CHECK(call) do {                                            \
    cudaError_t err = (call);                                            \
    if (err != cudaSuccess) {                                            \
        std::cerr << "CUDA error: " << cudaGetErrorString(err)           \
                  << " at " << __FILE__ << ":" << __LINE__ << "\n";      \
        std::exit(1);                                                    \
    }                                                                    \
} while(0)

// Ядро 1: коалесцированный доступ (соседние потоки -> соседние элементы)
__global__ void kernel_coalesced(const float* __restrict__ in,
                                 float* __restrict__ out,
                                 int n)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;   // глобальный индекс
    if (i < n)                                       // проверка границ
    {
        float x = in[i];                             // коалесцированное чтение
        out[i] = x * 2.0f + 1.0f;                    // простая операция
    }
}

// Ядро 2: некоалесцированный доступ (stride), соседние потоки читают далеко
__global__ void kernel_uncoalesced_stride(const float* __restrict__ in,
                                          float* __restrict__ out,
                                          int n,
                                          int stride)
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x; // глобальный tid
    int i = tid * stride;                            // индекс с шагом stride
    if (i < n)                                       // проверка границ
    {
        float x = in[i];                             // плохой паттерн чтения
        out[i] = x * 2.0f + 1.0f;                    // запись разреженная
    }
}

// Ядро 3: shared memory tile
// Сначала коалесцированно грузим блок в shared, потом считаем из shared
__global__ void kernel_shared_tiled(const float* __restrict__ in,
                                    float* __restrict__ out,
                                    int n)
{
    extern __shared__ float tile[];                  // динамическая shared память
    int i = blockIdx.x * blockDim.x + threadIdx.x;   // глобальный индекс
    int t = threadIdx.x;                             // локальный индекс

    if (i < n) tile[t] = in[i];                      // коалесцированная загрузка в shared
    else       tile[t] = 0.0f;                       // если вышли за границы

    __syncthreads();                                 // ждём загрузки всеми потоками

    float x = tile[t];                               // берём из shared
    float y = x * 2.0f + 1.0f;                       // считаем

    if (i < n) out[i] = y;                           // коалесцированная запись
}

// Эталон на CPU
static void cpu_ref(const std::vector<float>& in, std::vector<float>& out)
{
    for (size_t i = 0; i < in.size(); ++i)           // по всем элементам
        out[i] = in[i] * 2.0f + 1.0f;                // та же формула
}

// Замер времени для ЛЮБОГО launch-кода (лямбды) через cudaEvent
// Важно: здесь НЕТ <<< >>>, мы просто вызываем launch() внутри цикла
template <typename LaunchFn>
float time_launch(LaunchFn launch, int iters)
{
    cudaEvent_t start, stop;                         // события CUDA
    CUDA_CHECK(cudaEventCreate(&start));             // создаём start
    CUDA_CHECK(cudaEventCreate(&stop));              // создаём stop

    CUDA_CHECK(cudaDeviceSynchronize());             // синхронизируем перед замером
    CUDA_CHECK(cudaEventRecord(start));              // ставим старт

    for (int i = 0; i < iters; ++i)                  // повторяем много раз
        launch();                                     // запускаем то, что передали

    CUDA_CHECK(cudaEventRecord(stop));               // ставим стоп
    CUDA_CHECK(cudaEventSynchronize(stop));          // ждём стоп
    CUDA_CHECK(cudaGetLastError());                  // ловим ошибки ядра (если есть)

    float ms = 0.0f;                                 // миллисекунды
    CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop)); // считаем время

    CUDA_CHECK(cudaEventDestroy(start));             // удаляем события
    CUDA_CHECK(cudaEventDestroy(stop));

    return ms;                                       // время за iters запусков
}

int main(int argc, char** argv)
{
    int N = 1 << 24;                                 // по умолчанию ~16 млн
    if (argc > 1) N = std::atoi(argv[1]);            // N из аргументов
    if (N <= 0) { std::cout << "N должно быть > 0\n"; return 0; }

    int iters = 50;                                  // число повторов для стабильного времени
    int stride = 32;                                 // stride для плохого доступа

    // Данные на CPU
    std::vector<float> h_in(N);                      // вход
    std::vector<float> h_out(N, 0.0f);               // выход
    std::vector<float> h_ref(N, 0.0f);               // эталон

    for (int i = 0; i < N; ++i)                      // заполняем вход
        h_in[i] = (float)(i % 100) * 0.01f;          // простые повторяющиеся числа

    cpu_ref(h_in, h_ref);                            // эталон

    // Данные на GPU
    float* d_in = nullptr;                           // вход на GPU
    float* d_out = nullptr;                          // выход на GPU
    CUDA_CHECK(cudaMalloc(&d_in,  N * sizeof(float)));// malloc input
    CUDA_CHECK(cudaMalloc(&d_out, N * sizeof(float)));// malloc output

    CUDA_CHECK(cudaMemcpy(d_in, h_in.data(),
                          N * sizeof(float),
                          cudaMemcpyHostToDevice));  // копируем на GPU

    // Проверяем разные blockSize (организация потоков)
    std::vector<int> block_sizes = {128, 256, 512};   // варианты

    std::cout << "N=" << N << ", iters=" << iters << ", stride=" << stride << "\n";
    std::cout << "Columns: block, coalesced_ms, uncoalesced_ms, shared_ms\n";

    for (int bs : block_sizes)                        // перебор размеров блока
    {
        dim3 block(bs);                               // блок
        dim3 grid((N + bs - 1) / bs);                 // сетка для обычных ядер

        // 1) коалесцированное ядро
        auto launch_coal = [&]() {                    // лямбда запуска
            kernel_coalesced<<<grid, block>>>(d_in, d_out, N);
        };
        float ms_coal = time_launch(launch_coal, iters); // время за iters

        // 2) плохой доступ: ядро обрабатывает индексы i=tid*stride
        int n_bad = (N + stride - 1) / stride;        // сколько "полезных" tid
        dim3 grid_bad((n_bad + bs - 1) / bs);         // сетка для плохого ядра

        auto launch_uncoal = [&]() {                  // лямбда запуска
            kernel_uncoalesced_stride<<<grid_bad, block>>>(d_in, d_out, N, stride);
        };
        float ms_uncoal = time_launch(launch_uncoal, iters); // время

        // 3) shared memory ядро
        size_t shmem = (size_t)bs * sizeof(float);    // shared на блок
        auto launch_shared = [&]() {                  // лямбда запуска
            kernel_shared_tiled<<<grid, block, shmem>>>(d_in, d_out, N);
        };
        float ms_shared = time_launch(launch_shared, iters); // время

        // Печать среднего времени одного запуска (делим на iters)
        std::cout << bs << ", "
                  << (ms_coal / iters) << ", "
                  << (ms_uncoal / iters) << ", "
                  << (ms_shared / iters) << "\n";
    }

    // Проверка корректности (на коалесцированном ядре)
    {
        int bs = 256;                                 // block size для проверки
        dim3 block(bs);                               // block
        dim3 grid((N + bs - 1) / bs);                 // grid
        kernel_coalesced<<<grid, block>>>(d_in, d_out, N); // запуск
        CUDA_CHECK(cudaDeviceSynchronize());          // ждём
        CUDA_CHECK(cudaMemcpy(h_out.data(), d_out,
                              N * sizeof(float),
                              cudaMemcpyDeviceToHost)); // копируем назад

        double max_err = 0.0;                         // максимум ошибки
        for (int i = 0; i < N; ++i)                   // сравнение
        {
            double e = std::fabs((double)h_out[i] - (double)h_ref[i]); // ошибка
            if (e > max_err) max_err = e;             // обновили максимум
        }
        std::cout << "Max abs error (coalesced vs CPU ref) = " << max_err << "\n";
    }

    CUDA_CHECK(cudaFree(d_in));                       // освобождаем input
    CUDA_CHECK(cudaFree(d_out));                      // освобождаем output
    return 0;                                         // конец
}


Overwriting task2.cu


In [13]:
!nvcc -O2 -gencode arch=compute_75,code=sm_75 task2.cu -o task2
!./task2 8000000

N=8000000, iters=50, stride=32
Columns: block, coalesced_ms, uncoalesced_ms, shared_ms
128, 0.272735, 0.226547, 0.300196
256, 0.271114, 0.230157, 0.30593
512, 0.284988, 0.23511, 0.327761
Max abs error (coalesced vs CPU ref) = 0


In [ ]:
!./cuda_mem_patterns 10000000

**Задание 3.** Профилирование гибридного приложения CPU + GPU
Разработайте гибридную программу, в которой часть вычислений выполняется на CPU, а часть — на GPU.

**Требуется:**
1. реализовать гибридный алгоритм обработки массива данных;
2. использовать асинхронную передачу данных (cudaMemcpyAsync) и CUDA streams;
3. выполнить профилирование приложения:
a. определить накладные расходы передачи данных;
b. выявить узкие места при взаимодействии CPU и GPU;
4. предложить и реализовать одну оптимизацию, уменьшающую накладные расходы.

In [21]:
%%writefile task3.cu
// 1) Гибридная обработка массива: CPU считает часть, GPU считает часть (out[i] = in[i]*2 + 1)
// 2) Асинхронные копии cudaMemcpyAsync + CUDA streams + пайплайн по чанкам
// 3) Профилирование: отдельно считаем H2D / kernel / D2H / total и показываем узкие места
// 4) Оптимизация: pinned memory + overlap (две stream-ленты, двойная буферизация)

#include <cuda_runtime.h>     // CUDA runtime
#include <iostream>          // cout
#include <vector>            // vector
#include <cmath>             // fabs
#include <cstdlib>           // atoi, atoll
#include <chrono>            // chrono для CPU-тайминга
#include <algorithm>         // max
#include <cstring>

// Макрос проверки ошибок CUDA
#define CUDA_CHECK(call) do {                                                \
    cudaError_t err = (call);                                                \
    if (err != cudaSuccess) {                                                \
        std::cerr << "CUDA error: " << cudaGetErrorString(err)               \
                  << " at " << __FILE__ << ":" << __LINE__ << "\n";          \
        std::exit(1);                                                        \
    }                                                                        \
} while(0)

// Простое GPU-ядро: out[i] = in[i] * 2 + 1
__global__ void kernel_affine(const float* __restrict__ in,
                              float* __restrict__ out,
                              int n)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;   // глобальный индекс
    if (i < n)                                       // проверка границ
        out[i] = in[i] * 2.0f + 1.0f;                // вычисление
}

// CPU-референс (для проверки корректности)
static void cpu_ref(const float* in, float* out, int n)
{
    for (int i = 0; i < n; ++i)                      // по элементам
        out[i] = in[i] * 2.0f + 1.0f;                // то же самое
}

// CPU-часть гибридного алгоритма: считаем свою долю массива на CPU
static void cpu_process_part(const float* in, float* out, int begin, int end)
{
    for (int i = begin; i < end; ++i)                // диапазон CPU
        out[i] = in[i] * 2.0f + 1.0f;                // вычисление на CPU
}

// Вспомогательная печать времени (ms)
static double ms_between(std::chrono::high_resolution_clock::time_point a,
                         std::chrono::high_resolution_clock::time_point b)
{
    return std::chrono::duration<double, std::milli>(b - a).count();        // разница в мс
}

int main(int argc, char** argv)
{
    // параметры
    long long Nll = 16'000'000;                                            // размер массива по умолчанию
    if (argc > 1) Nll = std::atoll(argv[1]);                               // N из аргументов
    if (Nll <= 0) { std::cout << "N должно быть > 0\n"; return 0; }        // проверка
    if (Nll > (long long)2e9) { std::cout << "Слишком большой N\n"; return 0; } // защита

    int N = (int)Nll;                                                      // перевод в int для CUDA
    int cpu_share_percent = 25;                                            // сколько процентов обрабатывает CPU
    int cpu_n = (N * cpu_share_percent) / 100;                             // размер CPU-части
    int gpu_n = N - cpu_n;                                                 // размер GPU-части

    int block = 256;                                                       // размер блока CUDA
    int chunk = 1 << 20;                                                   // размер чанка для пайплайна (1,048,576 элементов)
    if (argc > 2) chunk = std::atoi(argv[2]);                              // chunk из аргументов
    if (chunk <= 0) { std::cout << "chunk должно быть > 0\n"; return 0; }  // проверка

    std::cout << "N=" << N << " elements\n";                               // печать N
    std::cout << "CPU share=" << cpu_share_percent << "% (" << cpu_n << ")\n";
    std::cout << "GPU share=" << (100 - cpu_share_percent) << "% (" << gpu_n << ")\n";
    std::cout << "chunk=" << chunk << " elements\n\n";

    //  входные данные (обычная host память)
    std::vector<float> h_in(N);                                            // вход на host
    std::vector<float> h_out_cpu(N, 0.0f);                                 // выход гибридный
    std::vector<float> h_out_ref(N, 0.0f);                                 // эталон

    for (int i = 0; i < N; ++i)                                            // заполняем вход
        h_in[i] = (float)(i % 100) * 0.01f;                                // повторяемые числа

    cpu_ref(h_in.data(), h_out_ref.data(), N);                             // эталон на CPU

    // ВАРИАНТ A: "база" — синхронные копии (cudaMemcpy) без overlap
    // Цель: получить накладные расходы передачи данных как baseline
    {
        std::cout << "=== Variant A (baseline): sync copies, no streams overlap ===\n";

        float* d_in = nullptr;                                             // device input
        float* d_out = nullptr;                                            // device output
        CUDA_CHECK(cudaMalloc(&d_in,  gpu_n * sizeof(float)));             // malloc GPU-часть
        CUDA_CHECK(cudaMalloc(&d_out, gpu_n * sizeof(float)));             // malloc GPU-часть

        // CPU таймер (полная длительность)
        auto tA0 = std::chrono::high_resolution_clock::now();              // старт total на CPU

        // 1) CPU считает свою часть (0..cpu_n)
        auto t_cpu0 = std::chrono::high_resolution_clock::now();           // старт CPU части
        cpu_process_part(h_in.data(), h_out_cpu.data(), 0, cpu_n);         // CPU вычисления
        auto t_cpu1 = std::chrono::high_resolution_clock::now();           // конец CPU части

        // 2) GPU считает оставшуюся часть (cpu_n..N) — копии синхронные
        // замерим H2D / kernel / D2H через cudaEvent
        cudaEvent_t e0, e1, e2, e3;                                        // события
        CUDA_CHECK(cudaEventCreate(&e0));                                  // create
        CUDA_CHECK(cudaEventCreate(&e1));
        CUDA_CHECK(cudaEventCreate(&e2));
        CUDA_CHECK(cudaEventCreate(&e3));

        CUDA_CHECK(cudaEventRecord(e0));                                   // отметка перед H2D
        CUDA_CHECK(cudaMemcpy(d_in, h_in.data() + cpu_n,                   // H2D (синхронно)
                              gpu_n * sizeof(float),
                              cudaMemcpyHostToDevice));
        CUDA_CHECK(cudaEventRecord(e1));                                   // отметка после H2D

        int grid = (gpu_n + block - 1) / block;                            // grid
        kernel_affine<<<grid, block>>>(d_in, d_out, gpu_n);                // kernel
        CUDA_CHECK(cudaGetLastError());                                    // проверка
        CUDA_CHECK(cudaEventRecord(e2));                                   // отметка после kernel

        CUDA_CHECK(cudaMemcpy(h_out_cpu.data() + cpu_n, d_out,            // D2H (синхронно)
                              gpu_n * sizeof(float),
                              cudaMemcpyDeviceToHost));
        CUDA_CHECK(cudaEventRecord(e3));                                   // отметка после D2H

        CUDA_CHECK(cudaEventSynchronize(e3));                              // ждём окончания всего
        auto tA1 = std::chrono::high_resolution_clock::now();              // конец total на CPU

        float ms_h2d=0, ms_k=0, ms_d2h=0;                                  // времена
        CUDA_CHECK(cudaEventElapsedTime(&ms_h2d, e0, e1));                 // H2D
        CUDA_CHECK(cudaEventElapsedTime(&ms_k,   e1, e2));                 // kernel (примерно)
        CUDA_CHECK(cudaEventElapsedTime(&ms_d2h, e2, e3));                 // D2H

        double ms_cpu = ms_between(t_cpu0, t_cpu1);                        // время CPU части
        double ms_total = ms_between(tA0, tA1);                            // полное время

        // Проверка корректности (макс. ошибка)
        double max_err = 0.0;                                              // максимум ошибки
        for (int i = 0; i < N; ++i)                                        // сравнение
            max_err = std::max(max_err, (double)std::fabs(h_out_cpu[i] - h_out_ref[i]));

        std::cout << "CPU part time (ms): " << ms_cpu << "\n";             // CPU время
        std::cout << "GPU H2D time (ms): " << ms_h2d << "\n";              // H2D
        std::cout << "GPU kernel time (ms): " << ms_k << "\n";             // kernel
        std::cout << "GPU D2H time (ms): " << ms_d2h << "\n";              // D2H
        std::cout << "TOTAL time (ms): " << ms_total << "\n";              // total
        std::cout << "Max abs error: " << max_err << "\n\n";               // корректность

        CUDA_CHECK(cudaEventDestroy(e0));                                  // destroy events
        CUDA_CHECK(cudaEventDestroy(e1));
        CUDA_CHECK(cudaEventDestroy(e2));
        CUDA_CHECK(cudaEventDestroy(e3));

        CUDA_CHECK(cudaFree(d_in));                                        // free
        CUDA_CHECK(cudaFree(d_out));
    }

    // ВАРИАНТ B: оптимизированный — pinned memory + cudaMemcpyAsync + streams
    // Идея: делим GPU-часть на чанки и делаем пайплайн (двойная буферизация)
    // Оптимизация (п.4): pinned memory + overlap H2D/kernel/D2H
    {
        std::cout << "=== Variant B (optimized): pinned + cudaMemcpyAsync + 2 streams (overlap) ===\n";

        // 1) Выделяем pinned память под GPU-часть входа/выхода (уменьшаем overhead копий)
        float* h_in_pinned = nullptr;                                      // pinned input для GPU-части
        float* h_out_pinned = nullptr;                                     // pinned output для GPU-части
        CUDA_CHECK(cudaMallocHost(&h_in_pinned,  gpu_n * sizeof(float)));  // pinned host input
        CUDA_CHECK(cudaMallocHost(&h_out_pinned, gpu_n * sizeof(float)));  // pinned host output

        // копируем данные GPU-части из vector в pinned (один раз)
        std::memcpy(h_in_pinned,  h_in.data()  + cpu_n, gpu_n * sizeof(float));
        std::memset(h_out_pinned, 0, gpu_n * sizeof(float));

        // 2) Две stream для двойной буферизации (ping-pong)
        cudaStream_t s0, s1;                                               // два потока CUDA
        CUDA_CHECK(cudaStreamCreate(&s0));                                 // create stream 0
        CUDA_CHECK(cudaStreamCreate(&s1));                                 // create stream 1

        // 3) Два device-буфера под чанки (ping-pong)
        float* d_in0=nullptr; float* d_out0=nullptr;                       // буферы для stream0
        float* d_in1=nullptr; float* d_out1=nullptr;                       // буферы для stream1
        CUDA_CHECK(cudaMalloc(&d_in0,  chunk * sizeof(float)));            // alloc chunk
        CUDA_CHECK(cudaMalloc(&d_out0, chunk * sizeof(float)));
        CUDA_CHECK(cudaMalloc(&d_in1,  chunk * sizeof(float)));
        CUDA_CHECK(cudaMalloc(&d_out1, chunk * sizeof(float)));

        // 4) События для профилирования (накладные передачи)
        cudaEvent_t e_start, e_end;                                        // события total GPU pipeline
        CUDA_CHECK(cudaEventCreate(&e_start));
        CUDA_CHECK(cudaEventCreate(&e_end));

        // CPU таймер полного времени
        auto tB0 = std::chrono::high_resolution_clock::now();              // старт total на CPU

        // CPU считает свою часть параллельно с GPU-пайплайном (на практике это и есть гибрид)
        auto t_cpu0 = std::chrono::high_resolution_clock::now();           // старт CPU части
        cpu_process_part(h_in.data(), h_out_cpu.data(), 0, cpu_n);         // CPU вычисления
        auto t_cpu1 = std::chrono::high_resolution_clock::now();           // конец CPU части

        CUDA_CHECK(cudaEventRecord(e_start, 0));                           // старт GPU-пайплайна (на default stream)

        // Для профилирования отдельно посчитаем суммарные времена H2D/D2H/Kernel по событиям на чанках
        double sum_h2d_ms = 0.0;                                           // суммарное время H2D
        double sum_d2h_ms = 0.0;                                           // суммарное время D2H
        double sum_k_ms   = 0.0;                                           // суммарное время kernel

        // Число чанков
        int chunks = (gpu_n + chunk - 1) / chunk;                          // сколько чанков

        // Для каждого чанка: async H2D -> kernel -> async D2H в своей stream
        for (int c = 0; c < chunks; ++c)                                   // по чанкам
        {
            int offset = c * chunk;                                        // смещение в GPU-части
            int cur = std::min(chunk, gpu_n - offset);                     // текущий размер чанка

            cudaStream_t stream = (c % 2 == 0) ? s0 : s1;                  // выбираем stream
            float* d_in  = (c % 2 == 0) ? d_in0  : d_in1;                  // выбираем device input
            float* d_out = (c % 2 == 0) ? d_out0 : d_out1;                 // выбираем device output

            // События на конкретный чанк, чтобы оценить стадии (создаём/удаляем быстро, но можно и пулом)
            cudaEvent_t eh0, eh1, ek0, ek1, ed0, ed1;                      // события стадий
            CUDA_CHECK(cudaEventCreate(&eh0)); CUDA_CHECK(cudaEventCreate(&eh1));
            CUDA_CHECK(cudaEventCreate(&ek0)); CUDA_CHECK(cudaEventCreate(&ek1));
            CUDA_CHECK(cudaEventCreate(&ed0)); CUDA_CHECK(cudaEventCreate(&ed1));

            CUDA_CHECK(cudaEventRecord(eh0, stream));                      // отметка H2D start
            CUDA_CHECK(cudaMemcpyAsync(d_in, h_in_pinned + offset,         // H2D async
                                       cur * sizeof(float),
                                       cudaMemcpyHostToDevice, stream));
            CUDA_CHECK(cudaEventRecord(eh1, stream));                      // отметка H2D end

            int grid = (cur + block - 1) / block;                          // grid для чанка

            CUDA_CHECK(cudaEventRecord(ek0, stream));                      // kernel start
            kernel_affine<<<grid, block, 0, stream>>>(d_in, d_out, cur);   // kernel в stream
            CUDA_CHECK(cudaGetLastError());                                // проверка
            CUDA_CHECK(cudaEventRecord(ek1, stream));                      // kernel end

            CUDA_CHECK(cudaEventRecord(ed0, stream));                      // D2H start
            CUDA_CHECK(cudaMemcpyAsync(h_out_pinned + offset, d_out,       // D2H async
                                       cur * sizeof(float),
                                       cudaMemcpyDeviceToHost, stream));
            CUDA_CHECK(cudaEventRecord(ed1, stream));                      // D2H end

            // Ждём завершение этого чанка, чтобы снять времена стадий (для профилирования)
            CUDA_CHECK(cudaEventSynchronize(ed1));                         // ждём конец D2H по чанку

            float h2d=0, k=0, d2h=0;                                       // локальные времена
            CUDA_CHECK(cudaEventElapsedTime(&h2d, eh0, eh1));              // H2D время чанка
            CUDA_CHECK(cudaEventElapsedTime(&k,   ek0, ek1));              // kernel время чанка
            CUDA_CHECK(cudaEventElapsedTime(&d2h, ed0, ed1));              // D2H время чанка

            sum_h2d_ms += h2d;                                             // суммируем
            sum_k_ms   += k;
            sum_d2h_ms += d2h;

            CUDA_CHECK(cudaEventDestroy(eh0)); CUDA_CHECK(cudaEventDestroy(eh1)); // destroy
            CUDA_CHECK(cudaEventDestroy(ek0)); CUDA_CHECK(cudaEventDestroy(ek1));
            CUDA_CHECK(cudaEventDestroy(ed0)); CUDA_CHECK(cudaEventDestroy(ed1));
        }

        // Дождёмся всех stream (на случай если что-то осталось)
        CUDA_CHECK(cudaStreamSynchronize(s0));                             // sync stream0
        CUDA_CHECK(cudaStreamSynchronize(s1));                             // sync stream1

        CUDA_CHECK(cudaEventRecord(e_end, 0));                             // конец пайплайна
        CUDA_CHECK(cudaEventSynchronize(e_end));                           // ждём

        float pipeline_ms = 0.0f;                                          // общее время GPU pipeline (по событию)
        CUDA_CHECK(cudaEventElapsedTime(&pipeline_ms, e_start, e_end));    // считаем

        // Копируем pinned output обратно в общий h_out_cpu в диапазон GPU-части
        std::memcpy(h_out_cpu.data() + cpu_n, h_out_pinned, gpu_n * sizeof(float));

        auto tB1 = std::chrono::high_resolution_clock::now();              // конец total на CPU

        double ms_cpu = ms_between(t_cpu0, t_cpu1);                        // CPU часть (внутри total)
        double ms_total = ms_between(tB0, tB1);                            // полное время программы

        // Проверка корректности (макс. ошибка)
        double max_err = 0.0;                                              // максимум ошибки
        for (int i = 0; i < N; ++i)                                        // сравнение
            max_err = std::max(max_err, (double)std::fabs(h_out_cpu[i] - h_out_ref[i]));

        std::cout << "CPU part time (ms): " << ms_cpu << "\n";             // CPU время
        std::cout << "GPU pipeline total (ms, overlapped): " << pipeline_ms << "\n";
        std::cout << "Sum H2D over chunks (ms): " << sum_h2d_ms << "\n";   // суммарно, без учёта overlap
        std::cout << "Sum kernel over chunks (ms): " << sum_k_ms << "\n";
        std::cout << "Sum D2H over chunks (ms): " << sum_d2h_ms << "\n";
        std::cout << "TOTAL time (ms): " << ms_total << "\n";              // total
        std::cout << "Max abs error: " << max_err << "\n\n";               // корректность

        // Убираем ресурсы
        CUDA_CHECK(cudaEventDestroy(e_start));                              // destroy events
        CUDA_CHECK(cudaEventDestroy(e_end));

        CUDA_CHECK(cudaFree(d_in0)); CUDA_CHECK(cudaFree(d_out0));          // free device buffers
        CUDA_CHECK(cudaFree(d_in1)); CUDA_CHECK(cudaFree(d_out1));

        CUDA_CHECK(cudaStreamDestroy(s0));                                  // destroy streams
        CUDA_CHECK(cudaStreamDestroy(s1));

        CUDA_CHECK(cudaFreeHost(h_in_pinned));                              // free pinned
        CUDA_CHECK(cudaFreeHost(h_out_pinned));
    }

    return 0;                                                               // конец
}

Overwriting task3.cu


In [22]:
!nvcc -O2 -arch=sm_75 task3.cu -o task3
!./task3 8000000 1048576

N=8000000 elements
CPU share=25% (2000000)
GPU share=75% (6000000)
chunk=1048576 elements

=== Variant A (baseline): sync copies, no streams overlap ===
CPU part time (ms): 1.54413
GPU H2D time (ms): 5.30106
GPU kernel time (ms): 0.244896
GPU D2H time (ms): 5.10989
TOTAL time (ms): 12.238
Max abs error: 0

=== Variant B (optimized): pinned + cudaMemcpyAsync + 2 streams (overlap) ===
CPU part time (ms): 2.02817
GPU pipeline total (ms, overlapped): 4.37411
Sum H2D over chunks (ms): 2.04186
Sum kernel over chunks (ms): 0.240544
Sum D2H over chunks (ms): 1.88282
TOTAL time (ms): 10.9521
Max abs error: 0



Для оценки накладных расходов передачи данных между CPU и GPU была выполнена детализация времени выполнения на этапы:

* вычисления на CPU;

* передача данных Host → Device (H2D);

* выполнение CUDA-ядра;

* передача данных Device → Host (D2H).

В базовой реализации (Variant A) использовались синхронные передачи данных (cudaMemcpy) без перекрытия вычислений и копирования.

В оптимизированной версии (Variant B) были применены следующие подходы:

* использование закреплённой (pinned) памяти на стороне host;

* асинхронные передачи данных (cudaMemcpyAsync);

* организация вычислений в двух CUDA streams с разбиением данных на чанки.

Полученные данные подтверждают, что использование pinned memory и асинхронных передач позволяет существенно сократить накладные расходы обмена данными.

Анализ профилирования показал, что в базовой реализации основным узким местом приложения является передача данных между CPU и GPU:

* суммарное время передач (H2D + D2H) составляет ≈ 10.41 мс;

* время выполнения вычислительного ядра на GPU составляет менее 0.3 мс.

Следовательно, приложение является memory-bound по передаче данных, а не вычислительно ограниченным. Производительность ограничивается пропускной способностью шины передачи данных между host и device, а не скоростью выполнения CUDA-ядра.

**Задание 4.** Анализ масштабируемости распределённой программы (MPI)
Реализуйте распределённую программу на MPI для вычисления агрегатной функции над
большим массивом (например, сумма, минимум, максимум).

**Требуется:**
* измерить время выполнения при различном числе процессов;
* оценить strong scaling и weak scaling;
* проанализировать влияние коммуникационных операций (MPI_Reduce,
MPI_Allreduce);
* сделать вывод о масштабируемости алгоритма и его практических ограничениях

In [23]:
%%writefile task4.cpp

#include <mpi.h>                 // MPI
#include <iostream>              // cout
#include <vector>                // vector
#include <random>                // mt19937
#include <limits>                // numeric_limits
#include <cstdlib>               // atoll, atoi
#include <algorithm>             // min, max

struct AggLocal                                                      // локальные агрегаты
{
    double sum;                                                      // сумма
    double minv;                                                     // минимум
    double maxv;                                                     // максимум
};

// заполнение локального массива случайными числами
static void fill_random(std::vector<double>& a, int seed)            // генерация
{
    std::mt19937 gen(seed);                                          // генератор
    std::uniform_real_distribution<double> dist(0.0, 1.0);           // 0..1
    for (size_t i = 0; i < a.size(); ++i)                            // цикл
        a[i] = dist(gen);                                            // значение
}

// вычисление локальных sum/min/max
static AggLocal local_agg(const std::vector<double>& a)              // локальная агрегация
{
    AggLocal r;                                                      // результат
    r.sum = 0.0;                                                     // старт суммы
    r.minv = std::numeric_limits<double>::infinity();                // старт минимума
    r.maxv = -std::numeric_limits<double>::infinity();               // старт максимума

    for (size_t i = 0; i < a.size(); ++i)                            // по массиву
    {
        double x = a[i];                                             // элемент
        r.sum += x;                                                  // суммируем
        if (x < r.minv) r.minv = x;                                  // минимум
        if (x > r.maxv) r.maxv = x;                                  // максимум
    }
    return r;                                                        // вернуть
}

int main(int argc, char** argv)                                     // вход
{
    MPI_Init(&argc, &argv);                                         // старт MPI

    int rank = 0;                                                   // rank
    int size = 0;                                                   // size
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);                           // rank
    MPI_Comm_size(MPI_COMM_WORLD, &size);                           // size

    // режимы:
    // strong: общий N фиксированный
    // weak: N_per_proc фиксированный, общий N = N_per_proc * size
    std::string mode = "strong";                                    // режим по умолчанию
    if (argc > 1) mode = argv[1];                                   // mode из argv[1]

    long long N_total = 50'000'000;                                 // общий N для strong
    long long N_per_proc = 10'000'000;                              // N на процесс для weak
    int use_allreduce = 0;                                          // 0 = Reduce, 1 = Allreduce

    if (mode == "strong")                                           // strong scaling
    {
        if (argc > 2) N_total = std::atoll(argv[2]);                // N_total
        if (argc > 3) use_allreduce = std::atoi(argv[3]);           // reduce/allreduce
    }
    else if (mode == "weak")                                        // weak scaling
    {
        if (argc > 2) N_per_proc = std::atoll(argv[2]);             // N_per_proc
        if (argc > 3) use_allreduce = std::atoi(argv[3]);           // reduce/allreduce
        N_total = N_per_proc * (long long)size;                     // общий N растёт с size
    }
    else                                                             // неверный режим
    {
        if (rank == 0)
        {
            std::cout << "Неверный режим. Используй:\n";
            std::cout << "  strong N_total use_allreduce(0/1)\n";
            std::cout << "  weak   N_per_proc use_allreduce(0/1)\n";
        }
        MPI_Finalize();                                             // конец MPI
        return 0;                                                   // выход
    }

    // распределение: делим N_total по процессам с остатком
    long long base = N_total / size;                                // базовый размер
    long long rem  = N_total % size;                                // остаток
    long long local_n = base + (rank < rem ? 1 : 0);                // размер локального куска

    // создаём локальный массив
    std::vector<double> local_data((size_t)local_n);                // локальные данные
    fill_random(local_data, 1234 + rank);                           // заполняем (seed разный на rank)

    MPI_Barrier(MPI_COMM_WORLD);                                    // синхронизация перед замером

    // отдельно меряем: compute_time и comm_time, и total
    double t0_total = MPI_Wtime();                                  // старт total

    double t0_comp = MPI_Wtime();                                   // старт compute
    AggLocal loc = local_agg(local_data);                           // локальная агрегация
    double t1_comp = MPI_Wtime();                                   // конец compute

    double compute_time = t1_comp - t0_comp;                        // время вычислений

    // глобальные агрегаты
    double gsum = 0.0;                                              // глобальная сумма
    double gmin = 0.0;                                              // глобальный минимум
    double gmax = 0.0;                                              // глобальный максимум

    double t0_comm = MPI_Wtime();                                   // старт коммуникаций

    if (use_allreduce == 0)                                         // вариант с MPI_Reduce
    {
        MPI_Reduce(&loc.sum,  &gsum, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD); // sum
        MPI_Reduce(&loc.minv, &gmin, 1, MPI_DOUBLE, MPI_MIN, 0, MPI_COMM_WORLD); // min
        MPI_Reduce(&loc.maxv, &gmax, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD); // max
    }
    else                                                            // вариант с MPI_Allreduce
    {
        MPI_Allreduce(&loc.sum,  &gsum, 1, MPI_DOUBLE, MPI_SUM, MPI_COMM_WORLD); // sum
        MPI_Allreduce(&loc.minv, &gmin, 1, MPI_DOUBLE, MPI_MIN, MPI_COMM_WORLD); // min
        MPI_Allreduce(&loc.maxv, &gmax, 1, MPI_DOUBLE, MPI_MAX, MPI_COMM_WORLD); // max
    }

    double t1_comm = MPI_Wtime();                                   // конец коммуникаций
    double comm_time = t1_comm - t0_comm;                           // время коммуникаций

    double t1_total = MPI_Wtime();                                  // конец total
    double total_time = t1_total - t0_total;                        // total время

    // берём максимальные времена по процессам (чтобы честно: кто самый медленный, тот и определяет время)
    double comp_max = 0.0;                                          // максимум compute
    double comm_max = 0.0;                                          // максимум comm
    double total_max = 0.0;                                         // максимум total

    MPI_Reduce(&compute_time, &comp_max, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD);
    MPI_Reduce(&comm_time,    &comm_max, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD);
    MPI_Reduce(&total_time,   &total_max,1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD);

    if (rank == 0)                                                  // вывод на root
    {
        std::string op = (use_allreduce == 0) ? "Reduce" : "Allreduce"; // что использовали

        std::cout << "mode=" << mode
                  << " procs=" << size
                  << " N_total=" << N_total
                  << " local_n~=" << base
                  << " op=" << op << "\n";

        std::cout << "time_total_s=" << total_max
                  << " time_compute_s=" << comp_max
                  << " time_comm_s=" << comm_max << "\n";

        // покажем результат агрегации (чисто для уверенности)
        std::cout << "sum=" << gsum << " min=" << gmin << " max=" << gmax << "\n";

        // доля коммуникаций
        double comm_share = (total_max > 0.0) ? (comm_max / total_max) : 0.0; // доля comm
        std::cout << "comm_share=" << comm_share << "\n\n";
    }

    MPI_Finalize();                                                  // конец MPI
    return 0;                                                        // выход
}

Writing task4.cpp


In [24]:
!mpic++ -O2 task4.cpp -o task4

In [25]:
!mpirun --allow-run-as-root --oversubscribe -np 1 ./task4 strong 50000000 0
!mpirun --allow-run-as-root --oversubscribe -np 2 ./task4 strong 50000000 0
!mpirun --allow-run-as-root --oversubscribe -np 4 ./task4 strong 50000000 0

mode=strong procs=1 N_total=50000000 local_n~=50000000 op=Reduce
time_total_s=0.0700232 time_compute_s=0.0700131 time_comm_s=9.331e-06
sum=2.49992e+07 min=1.79399e-08 max=1
comm_share=0.000133256

mode=strong procs=2 N_total=50000000 local_n~=25000000 op=Reduce
time_total_s=0.0505728 time_compute_s=0.0504933 time_comm_s=7.8862e-05
sum=2.49991e+07 min=6.95543e-09 max=1
comm_share=0.00155937

mode=strong procs=4 N_total=50000000 local_n~=12500000 op=Reduce
time_total_s=0.0517997 time_compute_s=0.0457352 time_comm_s=0.00606321
sum=2.50016e+07 min=5.48134e-11 max=1
comm_share=0.117051



In [26]:
!mpirun --allow-run-as-root --oversubscribe -np 1 ./task4 strong 50000000 1
!mpirun --allow-run-as-root --oversubscribe -np 2 ./task4 strong 50000000 1
!mpirun --allow-run-as-root --oversubscribe -np 4 ./task4 strong 50000000 1


mode=strong procs=1 N_total=50000000 local_n~=50000000 op=Allreduce
time_total_s=0.0709871 time_compute_s=0.0709765 time_comm_s=9.62e-06
sum=2.49992e+07 min=1.79399e-08 max=1
comm_share=0.000135518

mode=strong procs=2 N_total=50000000 local_n~=25000000 op=Allreduce
time_total_s=0.0503517 time_compute_s=0.0503095 time_comm_s=0.000177819
sum=2.49991e+07 min=6.95543e-09 max=1
comm_share=0.00353154

mode=strong procs=4 N_total=50000000 local_n~=12500000 op=Allreduce
time_total_s=0.0537572 time_compute_s=0.0513671 time_comm_s=0.0196138
sum=2.50016e+07 min=5.48134e-11 max=1
comm_share=0.364859

